In [1]:
import re
import pickle

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import tensorflow_hub as hub

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [5]:
# Загрузим датасет.
df = pd.read_csv('train.csv')

In [6]:
# Посмотрим на данные.
df.head()

pair_id                             name_1  \
0        1                Iko Industries Ltd.   
1        2            Apcotex Industries Ltd.   
2        3  Rishichem Distributors Pvt., Ltd.   
3        4            Powermax Rubber Factory   
4        5                          Tress A/S   

                                 name_2  is_duplicate  
0  Enormous Industrial Trade Pvt., Ltd.             0  
1   Technocraft Industries (India) Ltd.             0  
2                                   Dsa             0  
3                               Co. One             0  
4      Longyou Industries Park Zhejiang             0

In [7]:
df.shape

(497819, 4)

In [8]:
# Загрузим предобученный universal-sentence-encoder
use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [9]:
# Сделаем списки названий компаний.
df_brand_names = pd.DataFrame(pd.concat([df.name_1, df.name_2], axis=0).drop_duplicates(), columns=['name'])
brand_names = df_brand_names.name.tolist()

In [10]:
# Получим эмбеддинги из universal-sentence-encoder.
brand_embeddings = use(brand_names)

In [26]:
# Функция для нахождения топ 10 похожих на исходное названий.
def find_similar_name(sentence, embeddings):
    sentence_embedding = use([sentence])
    similarity = cosine_similarity(sentence_embedding, embeddings)
    top = sorted(enumerate(*similarity), key=lambda x: x[1], reverse=True)
    similar_names = [brand_names[x[0]] for x in top[:10]]
    for name in similar_names:
        print(name)    

In [27]:
# Найдём топ 10 похожих названий для компании Mediterranean Textile Co, которая была в датасете.
print(brand_names[80])
print()
find_similar_name(brand_names[80], brand_embeddings)

Mediterranean Textile Co.

Mediterranean Textile Co.
Zamoung Textile Co.
Torreh Textile Co.
Kavir Semnan Textile Co.
Ardakan Textile Co.
Ferdows Textile Co.
Hanam Textile Co.
Mediterranean Textile
Razi Abhar Textile Co.
Alroubaia Textile Co Fourtex


In [ ]:
# Вдими саму компанию и названия с такими же словами.

In [29]:
# Теперь получим эмбеддинг и найдём похожие названия для строки, которой не было в датасете.
my_text_test_1 = 'Darth Vader is a great Sith Lord'
print(my_text_test_1)
print()

find_similar_name(my_text_test_1, brand_embeddings)

Darth Vader is a great Sith Lord

Hussein Hussein Hezam Mohmmed
Sonic Fluid Power Systems (Pvt.)
Atat Tasimacilik Cozumleri A.S.
Abdullah Hussain Ali
The Cato Corporation
Ashoka Inc.
Sound Merch Bmg (Neutral)
Hyundai Power Transformers Usa
Sika S.A. Chile
Kia Chile S.A.


In [ ]:
# Интересно, что к Дарту Вейдеру модель вывела названия, где фигурировало слово Power. Аж два названия.
# "Atat" возможно из-за похожести на AT-AT (Имперский шагоход). 
# И "Ashoka Inc." из-за похожести  "Ashoka" на "Ahsoka". 
# Потому что Ahsoka Tano это ученица Вейдера, времён, когда его еще Энакином звали.

In [31]:
# Теперь получим эмбеддинг и найдём похожие названия для строки, похожей, на то, что было в датасете.
# В датасете было название "Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V."
# Посмотрим, что на него выдаст модель.

my_text_test_2 = 'Well Aero System Lab Industrial Craft'
print(my_text_test_2)
print()

find_similar_name(my_text_test_2, brand_embeddings)

Well Aero System Lab Industrial Craft

Golden Wheel Industrial Mfg. Corp
Titan Rubber Industrial Mfg. Corp
Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Mova Industrial S.A.C.
Newpro Industrial Mfg. Corp
Aero Club
Erbus Industrial Eireli
Aero Traders
Distribuidora Industrial Avanzada Sa De Cv
Pegsa Industrial Sac


In [ ]:
# Видим, что нашлись названия компаний с похожими словами:  "Industrial", "Aero", "Aerospace", "Laboratory".  